In [121]:
from sympy.series import series
from scipy.optimize import minimize, NonlinearConstraint
import time
import itertools
from sympy import symbols, Matrix, SparseMatrix, cos, sin, expand, lambdify, O
from sympy.utilities.iterables import multiset_permutations

from functools import *
from operator import *

import sympy
import numpy as np
import random
import math



# Stuff from Tobias' code

In [ ]:
class interactions:
    def __init__(self, N , inter):
        self.N = N
        self.interactions = inter
        
        
    def split(self):
        paulis = []
        positions = []
        for i in range(len(self.interactions)):
            pauli_lst = []
            pos_lst = []

        
            #creating lists of operators and corresponding positions
            prev_int = False
            for k in self.interactions[i][0]:
                if k.isdigit():
                    if not prev_int:
                        pos_lst.append(k)
                    else:
                        pos_lst[-1] += k
                    prev_int = True
                else:
                    pauli_lst.append(k)
                    prev_int = False
            paulis.append(pauli_lst)
            positions.append(pos_lst)
        return positions, paulis
    
    
    
    def Hamiltonian(self):
        H = np.zeros((2**self.N,2**self.N)).astype('complex128')
        positions, paulis = self.split()
        for i in range(len(self.interactions)):
            Kron = 1

            #computes tensorproduct for every interaction
            n=0
            for j in range(self.N):

                if j+1 == int(positions[i][n]):
                    M = eval(paulis[i][n])
                    if n+1 < len(positions[i]):
                        n+=1
                else:
                    M = I
                print(M)
                
                Kron = np.kron(Kron,M)
            print(self.interactions[i][1],Kron)
            H += Kron*self.interactions[i][1]
        Z_terms = -1*sum([reduce(np.kron,[Z if j == i else I for j in range(self.N)]) for i in range(self.N)])
        H +=Z_terms
        return H
    
    def exact_eigen(self):
        H = self.Hamiltonian()
        eigen = np.linalg.eig(H)
        index = np.argmin(eigen[0])
        ground_energy = eigen[0][index]
        ground_state = eigen[1][:,index]
        return ground_state, ground_energy

    
def single_pauli_action(pauli, spin):
    
    if pauli=='X':
        return((spin+1)%2, 1)
    elif pauli=='Y':
        return((spin+1)%2, 1j*(-1)**spin)
    elif pauli=='Z':
        return(spin, (-1)**spin)
    elif pauli=='I':
        return(spin, 1)
    else:
        print('wrong pauli!')
        return(None)
    
def state_k(N,generators,k,starting_state = np.zeros(N)):
    positions, paulis = interactions(N, generators).split() 
    a = 1
    init_state = starting_state.copy()
    for i in range(len(k)):
        if k[i] %2 != 0:
            for j in range(len(positions[i])):
                pauli = paulis[i][j]
                spin = init_state[int(positions[i][j])-1]
                new_spin, factor = single_pauli_action(pauli,spin)
                init_state[int(positions[i][j])-1] = new_spin
                a *= factor
                



    state = init_state
    state = [int(i) for i in state]
    return a, list(state)


def findCombinationsUtil(li, arr, index, num, reducedNum):
    z = []
    if (reducedNum < 0): 
        return; 
    if (reducedNum == 0): 
  
        for i in range(index): 
            z = z + [arr[i]]
        li.append(z) 
        return;

    prev = 1 if (index == 0) else arr[index - 1]; 
  
    for k in range(prev, num + 1): 
          

        arr[index] = k; 
  
        findCombinationsUtil(li,arr, index + 1, num,  
                                 reducedNum - k); 
    return li

def k_all(N, generators, order): 
      
    # array to store the combinations 
    # It can contain max n elements
    out = []
    k_length = len(generators)
    for k in range(1, order+1):
        arr = [0] * k;
        output = []
        a =  findCombinationsUtil([], arr, 0, k, k);
        for i in a:
            if len(i)<= k_length:
                i = i.extend((k_length-len(i))*[0])
        for j in a:
            if len(j) == k_length:
#                 if k_vector(N, interactions,j).state()[1] != N*[0]:
                output = output + list(multiset_permutations(j))
        out =  out+ output
    return [[0]*k_length]+out

def power_product(x,y):
    out = 1
    for i in range(len(x)):
         out*= x[i]**y[i]
    return out

# Quantum circuit PT

## Code for Hamiltonians and ansatzes: generalities, TFIM Hamiltonian, and TFIM QAOA

In [325]:
def mb_op(P):
    
    P_1b_ops=[Paulis[p] for p in P]
    
    P_mb_op=reduce(np.kron, P_1b_ops)
    
    return(P_mb_op)






def P_str_exp(P, alpha):
    
    P_mb_op=mb_op(P)
    
    identity=np.eye(2**len(P))
    
    if str(type(alpha))[8:-2]=='sympy.core.symbol.Symbol':
    
        exp_P_op=cos(alpha)*SparseMatrix(identity)+1j*sin(alpha)*SparseMatrix(P_mb_op)
        
    else:
        
        exp_P_op=np.cos(alpha)*identity+1j*np.sin(alpha)*P_mb_op
    
    return(exp_P_op)

def Ansatz_U(ansatz_Ps, thetas):
    
    U=np.eye(2**size)
        
    if ansatz_Ps!=[]:

        U=reduce(lambda x,y: y@x, [P_str_exp(P, thetas[l]) for l, P in enumerate(ansatz_Ps)])@U
        
    
    return(U)


In [326]:



def TFIM_Hamiltonian_generators(size,J):



    generators=[]

    tracker=0

    for _ in range(p):

        for site in range(size-1):

            generators.append([f'X{site}X{site+1}', J])

            tracker+=1

        for site in range(size):

            generators.append([f'Z{site}', -1])
            
    return(generators)

def TFIM(size, g):
    
    
    
    XX_Ps=[['X' if (site2==site1+1)|(site2==site1) else 'I' for site2 in range(size)] 
           for site1 in range(size-1)]
    
    Z_Ps=[['Z' if (site2==site1) else 'I' for site2 in range(size)] 
           for site1 in range(size)]
    
    H=-sum([mb_op(Z_P) for Z_P in Z_Ps])+g*sum([mb_op(XX_P) for XX_P in XX_Ps])
    
    return(H)





In [327]:
def TFIM_QAOA_generators(size,p, thetas):


    generators=[]

    tracker=0

    for _ in range(p):

        for site in range(size-1):

            generators.append([f'X{site}X{site+1}', thetas[tracker]])

            tracker+=1

        for site in range(size):

            generators.append([f'Z{site}', thetas[tracker]])
            
    return(generators)

def TFIM_QAOA(size, p, thetas):
    
    assert (p*(2*size-1)==len(thetas))
    
    XX_Ps=[['X' if (site2==site1+1)|(site2==site1) else 'I' for site2 in range(size)] 
           for site1 in range(size-1)]
    
    Z_Ps=[['Z' if (site2==site1) else 'I' for site2 in range(size)] 
           for site1 in range(size)]
    
    ansatz_Ps=[]
    
    for _ in range(p):
        
        ansatz_Ps+=XX_Ps
        
        ansatz_Ps+=Z_Ps
        
        
    U=Ansatz_U(ansatz_Ps, thetas)
    
    return (U)




## Generating $E(\phi,\vec{k})$ for TFIM & QAOA : code by Tobias but improved

In [598]:
N = 4
J = 2
order = 4
p=1
num_pars=(2*N-1)*p

In [602]:
stopwatch=time.time()



thetas=symbols(reduce(add, [f'θ{i}, ' for i in range(num_pars)])[:-2],real=True)

phis=symbols(reduce(add, [f'φ{i}, ' for i in range(num_pars)])[:-2],real=True)



if num_pars==1:
    thetas=(thetas,)

    phis=(phis,)

epsilon=symbols('ε')

theta_subs=tuple(thetas[i]*epsilon for i in range(num_pars))


    

generators = TFIM_QAOA_generators(N,p,thetas)


H = TFIM_Hamiltonian_generators(N,J)


# generators = [["X1X2",thetas[2]],["Z1",thetas[0]],["Z2",thetas[1]]]


# H = [["Z1",-1],["Z2",-1],["X1X2",J]]



K_all = k_all(N, generators, order)





###################################################





lst = list(map(list, itertools.product([0, 1], repeat=N)))


s_dict = {str(i):{rdr: 0 for rdr in range(order+1)} for i in lst}
# print(s_dict)

for k in K_all:
    G, s = state_k(N,generators,k,np.zeros(N))
    thetas_prod = power_product(list(thetas), k)
    factorial = np.prod(np.array([math.factorial(i) for i in k]))
#     print(s,G)
    s_dict[str(s)][sum(k)] += thetas_prod*(1j)**sum(k)*G/factorial
# print(s_dict)


Hs = {str(i):{rdr: 0 for rdr in range(order+1)} for i in lst}
for i in lst:
    for j in range(len(H)):
        k = [0]*len(H)
        k[j] = 1

        for rdr in range(order+1):
        
            factor = s_dict[str(i)][rdr]

            if factor != 0:

                sign, state = state_k(N, H, k, i)

                if s_dict[str(state)] != 0:

                    factor*=sign*H[j][1]
                    Hs[str(state)][rdr] += factor

        
        
#             print(sign,state, H[j],i,factor)
# print(Hs)


E = 0
for i in lst:
    
    
    for rdr1 in range(order+1):
    
        if Hs[str(i)][rdr1]!=0:
    
            for rdr2 in range(order+1-rdr1):
                
                if s_dict[str(i)][rdr2]!=0:
                
                    E += Hs[str(i)][rdr1]*np.conj(s_dict[str(i)][rdr2])

E=expand(E)

time.time()-stopwatch

1.0921661853790283

In [603]:
E

-1.33333333333333*θ0**4 - 5.33333333333333*θ0**3*θ3 - 5.33333333333333*θ0**3*θ4 - 4.0*θ0**2*θ1**2 - 16.0*θ0**2*θ1*θ4 + 4.0*θ0**2 - 16.0*θ0*θ1**2*θ4 - 5.33333333333333*θ0*θ3**3 - 16.0*θ0*θ3**2*θ4 - 16.0*θ0*θ3*θ4**2 + 8.0*θ0*θ3 - 5.33333333333333*θ0*θ4**3 + 8.0*θ0*θ4 - 5.55111512312578e-17*θ0*θ5**3 - 5.55111512312578e-17*θ0*θ6**3 - 1.33333333333333*θ1**4 - 5.33333333333333*θ1**3*θ4 - 5.33333333333333*θ1**3*θ5 - 4.0*θ1**2*θ2**2 - 16.0*θ1**2*θ2*θ5 + 4.0*θ1**2 - 16.0*θ1*θ2**2*θ5 - 5.55111512312578e-17*θ1*θ3**3 - 5.33333333333333*θ1*θ4**3 - 16.0*θ1*θ4**2*θ5 - 16.0*θ1*θ4*θ5**2 + 8.0*θ1*θ4 - 5.33333333333333*θ1*θ5**3 + 8.0*θ1*θ5 - 5.55111512312578e-17*θ1*θ6**3 - 1.33333333333333*θ2**4 - 5.33333333333333*θ2**3*θ5 - 5.33333333333333*θ2**3*θ6 + 4.0*θ2**2 - 5.55111512312578e-17*θ2*θ3**3 - 5.55111512312578e-17*θ2*θ4**3 - 5.33333333333333*θ2*θ5**3 - 16.0*θ2*θ5**2*θ6 - 16.0*θ2*θ5*θ6**2 + 8.0*θ2*θ5 - 5.33333333333333*θ2*θ6**3 + 8.0*θ2*θ6 - 4.0

This basically works, now need to add Cliffords

*Scaling seems to be N^4 with the number of qubits, D^2 with the depth*

## Cell search via full simulation

## Method explanation on a single-qubit example

Consider a single spin example first. The Hamiltonian is this:

\begin{equation}
    H=-(\cos\eta \,Z + \sin\eta \,X),
\end{equation}

and the parameterizing circuit is this:

\begin{equation}
    U=e^{i\theta Y}.
\end{equation}

I then want to create a function $E(\phi, k)$, where $k \in \{-1,0,1,2\}$, and $U$ expanded as follows:

\begin{equation}
    U=e^{i\phi + i\frac{K\pi}{4}}
\end{equation}

For this, I need first a function $|\psi\rangle(\phi, k)$. I can code it via direct simulation first (with full circuit & Hamiltonian matrices, with sines and cosines etc). Obtaining $E(\phi, k)$ in this full-blown way, one can then Taylor-expand it to an even order $n$. Then the optimal spot $\phi=\Phi^\text{ (n)}_{k}$ is to be found by a minimization procedure, with bounds $\Phi^\text{ (n)}_{k}\in(-\pi/4,\pi/4)$

With no info, one can start from $k=0$ Clifford cell and use expansion to $2$nd order in $\phi$, corresponding to $n=2$. After the optimum $\phi^\text{ (2)}_{k=0}$ is identified, there are two possible outcomes:

* $|\Phi^\text{(2)}_{0}| < \pi/8$: in this case clearly $k=0$ is the right Clifford cell. The algorithm can be terminated, and we can use $\Phi^\text{ (2)}_{0}$ as the starting point $\Phi$.
* $|\Phi^\text{(2)}_{0}| > \pi/8$: in this case $k=\pm 1$ might be preferred and needs to be checked as well. 

This search for optimal $k$ and $\phi$ can be continued like this, until a specific $k=K \in \{-1,0,1,2\}$ is found, such that $|\Phi^\text{ (2)}_{K}| < \pi/8$. In this case, one can use $\Phi=\Phi^\text{ (2)}_{K}$ as the optimal point.

It may happen that no such optimal cell $K$ is found, and instead one loops over cells. E.g. this can happen if the actual optimal angle is around $\pi/8$. In this case there will be two competing cells $\{K_1, K_2=K_1+1\}$ for which either simultaneously $|\Phi^\text{ (2)}_{K_i}|>\pi/8$. In this situation, one can follow one of the two routes:

* Fix the starting point as $K_1,\Phi=\pi/8$, or more practically, as the arithmetic average $K_1, \Phi=\frac{1}{2}(\Phi^\text{ (2)}_{K_1}+\Phi^\text{ (2)}_{K_2}+\pi/8)$.
* Increase the expansion degree $n$ from $2$ to $4$, which will allow to resolve between $K_1$ and $K_2$.

The latter approach is the only reliable method, since it can be continued indefinitely (going to $6$ and beyond) while recovering the entire function. However it does not scale well. If the true angle is close or exactly equals to $\pi/8$, the degree $n$ required for perfect resolution will diverge. However if $\frac{1}{2}(\Phi^\text{ (2)}_{K_1}+\Phi^\text{ (2)}_{K_2}+\pi/8)$ is close enough to $\pi/8$ (with some tolerance criteria), one may be willing to accept it as a good enough approximation.

### Complications added in the many-body case 

*To be written up*

### Code structure



Functions and variables:

* thetas, phis, k -- self-explanatory; phi renormalized like K, s.t. pi/8 corresponds to 1/2
* Psi, H, E -- numpy/sympy hybrids
* E_expand -- function to find an expanded expression
* E_evaluate -- function to evaluate energy

Structure of the algorithm:

0. Have a running K_0, n_0=2
1. Find optimal $\Phi^\text{(2)}_{0}$
2. Store into the 
3. Either END, or suggest next K_0

### Code preliminaries

In [25]:
def angle_expand(Q, thetas, phis, k, n=2, theta_subs=None):
    
    
    
    num_pars=len(thetas)
    
    epsilon=symbols('epsilon')
    
    
    
    if theta_subs==None:

        theta_subs=tuple((phis[i]*epsilon+k[i])*np.pi/4 for i in range(num_pars))

    Q_phi=Q.subs({theta: theta_subs[i] for i, theta in enumerate(thetas)})
    
    if n==None:
        return Q_phi.subs(epsilon,1)
    
    Q_phi=expand(series(Q_phi, epsilon, x0=0, n=n+1)).removeO().subs(epsilon,1)
    
    return (Q_phi)

def angle_truncate(Q, thetas, n=2, theta_subs=None, epsilon=symbols('ε')):
    
    num_pars=len(thetas)
    
    if theta_subs==None:

        theta_subs=tuple(thetas[i]*epsilon for i in range(num_pars))


    Q_tr=(expand(Q.subs({theta: theta_subs[i] for i, theta in enumerate(thetas)}))+O(epsilon**(n+1))).removeO().subs(epsilon,1)
    
    return (Q_tr)


# def E_evaluate(E_expanded, phis, phi_subs):
         
               
#     return (float(E_expanded.subs({phi:phi_subs[i] for i, phi in enumerate(phis)})))







In [284]:
def perturbative_optimum (E, thetas, phis, k0=None, n=2, maxiter=16):
    
    
    if k0==None:
        
        k0=tuple(0 for _ in range(len(thetas)))
    
    num_pars=len(thetas)
    
    k=k0
#     time_spent_in_min=0
    
#     time_spent_in_exp=0
    
    optima_history={'k':[], 'Phi':[]}
    
    for iteration in range(maxiter):
        
        ###################################################
        
        
        if optima_history['k'].count(k) == 3 :
            
            #means that this cell was already explored previously. so if 
            #procedure is continued, one would loop between the cells
            #which occured since then, indefinitely.
            #so, we decide for the optimal point based on the procedure
            #defined above (without increasing the approximation degree)
            
        
            loop_start = optima_history['k'].index(k)
            
            optima_history['k']=optima_history['k'][loop_start:]
            
            optima_history['Phi']=optima_history['Phi'][loop_start:]
            
            print(f'Triple loop of size {len(optima_history["k"])} encountered')

            
            acceptable_cells=[(np.abs(Phi)<0.5).all() for Phi in optima_history['Phi']]

            if acceptable_cells.count(True)==1:

                print(f'Only one cell yields consistent approximation, outputting that')  

                correct_cell_number=acceptable_cells.index(True)

                k=optima_history['k'][correct_cell_number]

                Phi=optima_history['Phi'][correct_cell_number]
                  
                  
                ####  
                E_expanded=angle_expand(E, thetas, phis, k, n)

                E_eval=lambdify(phis, E_expanded)

                break
                  
                ###

            elif acceptable_cells.count(True)>1:

                print(f'Multiple cells yield consistent approximation; discarding the rest from the loop')

                optima_history['k']=[k for k, index in enumerate(optima_history['k']) 
                                     if acceptable_cells[index]]

                optima_history['Phi']=[Phi for Phi, index in enumerate(optima_history['Phi']) 
                                     if acceptable_cells[index]]
            else:

                print(f'No cell yields consistent approximation, using all of them for the output')

            print('Outputting the mean k and Phi across the loop')

            mean_theta=np.mean([np.array(k) for k in optima_history['k']],axis=0)

            mean_theta+=np.mean([np.array(a_Phi) for a_Phi in optima_history['Phi']],axis=0)

            print(mean_theta)

            k=np.round(mean_theta).astype('int')

            Phi=mean_theta-k

            k=tuple(k)

            Phi=tuple(Phi)
                  
            ###
                  
            E_expanded=angle_expand(E, thetas, phis, k, n)

            E_eval=lambdify(phis, E_expanded)

            break
            
            ###
        
        ###################################################          
                  
                  
#         stopwatch=time.time()
                  
        E_expanded=angle_expand(E, thetas, phis, k, n)
        
#         time_spent_in_exp+=time.time()-stopwatch 
        
        E_eval=lambdify(phis, E_expanded)          
        
#         stopwatch=time.time()          
        
        print(f'Value at the center of the cell: {E_eval(*[0 for _ in range(num_pars)])}')
        
                  
        def cons_f(phis):
    
            return [sum(phis**2)]

        def cons_J(phis):

            return(2*phis)

        def cons_H(phis,v):

            return(   np.eye(len(phis))*2*v   )

        nl_cons=[NonlinearConstraint(cons_f, -np.inf, (0.5**2)*len(phis), jac=cons_J, hess=cons_H)]
                  
        if n==None:
                  
            nl_cons=tuple()
                  
                  
        min_results=minimize(lambda phi0: E_eval(*phi0), 
                             np.zeros(num_pars)+np.random.random(num_pars)*0.01, method='trust-constr',
                             constraints=nl_cons)
        
#         time_spent_in_min+=time.time()-stopwatch          
                  
        Phi=min_results.x
                  
        nfev=min_results.nfev
                  
        
        
        print(f'Trying cell k={k}. Found optimal point Phi={Phi} in {nfev} function calls')
        
        print(f'Optimal energy is {min_results.fun}')
                  
#         print(f'Full minimization data is as follows: {min_results}')
                  
        ###################################################          
        
        if n!=None:
            cell_exits=np.abs(Phi)>0.5
        else:
            cell_exits=np.array([False])
        
        if cell_exits.any(): 
            
            
            

            optima_history['k'].append(tuple(k))
            optima_history['Phi'].append(tuple(Phi))
            
            exit_indices=[ind for ind, phi_ind in enumerate(Phi) if np.abs(phi_ind)==max(np.abs(Phi))]
            
            exit_direction=random.choice(exit_indices)
                  
            exit_direction=np.array([1 if d==exit_direction else 0 
                                     for d in range(len(cell_exits))])
                  
            corrections=exit_direction*np.sign(Phi).astype('int')
            
            k+=corrections
                  
            k=tuple(k)    
            
            print(f'Phi is outside the right cell, constraint saturation {len(Phi)*0.25-sum(Phi**2)}')
                  
            print(f'Moving to the corrected cell..')
            
            
            
        else:
            
            
            print(f'Phi is inside the right cell, finalizing procedure, constraint saturation {len(Phi)*0.25-sum(Phi**2)}')
            
            break
    
                  
 
                  
    print(f'The resulting angle theta is: {np.array(k)+np.array(Phi)}, phi is {np.array(Phi)}, the estimated energy is {E_eval(*Phi)}')
#     print(f'Total time spent in min, exp:{time_spent_in_min, time_spent_in_exp}')
    
    return({'K': k, 'Phi': Phi}, optima_history)
    

Generating Hamiltonian

In [157]:
Paulis={'X': np.array([[0,1],[1,0]], dtype='complex'), 
        'Y': np.array([[0,-1j],[1j,0]], dtype='complex'),
        'Z': np.array([[1,0],[0,-1]], dtype='complex'),
        'I': np.array([[1,0],[0,1]], dtype='complex')}

### TFIM QAOA application

In [315]:
size=3

g=10

p=1

num_pars=(2*size-1)*p

stopwatch=time.time()

# num_pars=7

thetas=symbols(reduce(add, [f'θ{i}, ' for i in range(num_pars)])[:-2],real=True)

phis=symbols(reduce(add, [f'φ{i}, ' for i in range(num_pars)])[:-2],real=True)


if num_pars==1:
    thetas=(thetas,)

    phis=(phis,)
    

U= TFIM_QAOA(size, p, thetas)

psi=U@SparseMatrix([1]+[0 for _ in range(2**size-1)])
   

H = TFIM(size,g)

E_GS=np.linalg.eigh(H)[0][0]

E=(expand((np.conj(psi.T)@H@psi)[0]))

# E=E_expand(E,thetas,phis,[0 for _ in range(num_pars)])


time.time()-stopwatch

0.382373571395874

In [329]:
E

1.0*sin(θ0)**2*sin(θ1)**2*sin(θ2)**2*sin(θ3)**2*sin(θ4)**2 + 1.0*sin(θ0)**2*sin(θ1)**2*sin(θ2)**2*sin(θ3)**2*cos(θ4)**2 + 1.0*sin(θ0)**2*sin(θ1)**2*sin(θ2)**2*sin(θ4)**2*cos(θ3)**2 + 1.0*sin(θ0)**2*sin(θ1)**2*sin(θ2)**2*cos(θ3)**2*cos(θ4)**2 + 1.0*sin(θ0)**2*sin(θ1)**2*sin(θ3)**2*sin(θ4)**2*cos(θ2)**2 + 1.0*sin(θ0)**2*sin(θ1)**2*sin(θ3)**2*cos(θ2)**2*cos(θ4)**2 + 1.0*sin(θ0)**2*sin(θ1)**2*sin(θ4)**2*cos(θ2)**2*cos(θ3)**2 + 1.0*sin(θ0)**2*sin(θ1)**2*cos(θ2)**2*cos(θ3)**2*cos(θ4)**2 - 40.0*sin(θ0)**2*sin(θ1)*sin(θ2)**2*sin(θ3)**2*sin(θ4)*cos(θ1)*cos(θ4) + 40.0*sin(θ0)**2*sin(θ1)*sin(θ2)**2*sin(θ3)*sin(θ4)**2*cos(θ1)*cos(θ3) - 40.0*sin(θ0)**2*sin(θ1)*sin(θ2)**2*sin(θ3)*cos(θ1)*cos(θ3)*cos(θ4)**2 + 40.0*sin(θ0)**2*sin(θ1)*sin(θ2)**2*sin(θ4)*cos(θ1)*cos(θ3)**2*cos(θ4) - 40.0*sin(θ0)**2*sin(θ1)*sin(θ3)**2*sin(θ4)*cos(θ1)*cos(θ2)**2*cos(θ4) + 40.0*sin(θ0)**2*sin(θ1)*sin(θ3)*sin(θ4)**2*cos(θ1)*cos(θ2)**2*cos(θ3) - 40.0*sin(θ0)**2*sin(θ1)*sin(θ3)*cos(θ1)*cos(θ2)**2*cos(θ3)*cos(θ4)**2 + 40.0*sin

In [59]:
E_GS

-4.806423851823103

In [317]:
def cons_f(phi):
    
    return [sum(phi**2)]

def cons_J(phi):
    
    return(2*phi)

def cons_H(phi,v):
    
    return(2*np.diag(v))

nl_cons=NonlinearConstraint(cons_f, -np.inf, 2, jac=cons_J, hess=BFGS())

nl_cons=tuple()

minimize(lambda phi: sum([(phi_l-3)**2 for phi_l in phi]), [0,0], method='trust-constr', constraints=nl_cons)

         cg_niter: 3
     cg_stop_cond: 4
           constr: []
      constr_nfev: []
      constr_nhev: []
      constr_njev: []
   constr_penalty: 1.0
 constr_violation: 0
   execution_time: 0.027634143829345703
              fun: 9.992006824579719e-16
             grad: array([4.4408921e-16, 4.4408921e-16])
              jac: []
  lagrangian_grad: array([4.4408921e-16, 4.4408921e-16])
          message: '`gtol` termination condition is satisfied.'
           method: 'equality_constrained_sqp'
             nfev: 12
             nhev: 0
              nit: 4
            niter: 4
             njev: 4
       optimality: 4.440892098500626e-16
           status: 1
          success: True
        tr_radius: 22.69848531810293
                v: []
                x: array([2.99999998, 2.99999998])

In [323]:
stopwatch=time.time()

perturbative_optimum(E,thetas,phis, k0=[0, 0, 0, 0, 0], n=None, maxiter=15)

print(f'TOTAL TIME OVERALL: {time.time()-stopwatch}')

Value at the center of the cell: -3.0
Trying cell k=[0, 0, 0, 0, 0]. Found optimal point Phi=[ 9.29600811e-01 -9.29600824e-01 -9.99999978e-01  1.17431961e-09
  1.00000000e+00] in 186 function calls
Optimal energy is -20.11073291785114
Phi is inside the right cell, finalizing procedure, constraint saturation -2.4783153139541314
The resulting angle theta is: [ 9.29600811e-01 -9.29600824e-01 -9.99999978e-01  1.17431961e-09
  1.00000000e+00], phi is [ 9.29600811e-01 -9.29600824e-01 -9.99999978e-01  1.17431961e-09
  1.00000000e+00], the estimated energy is -20.11073291785114
TOTAL TIME OVERALL: 1.7364215850830078
